In [1]:
!pip install torch librosa scikit-learn 'transformers[torch]' datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 3.7 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 11.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 11.4 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 4.6 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 9.1 MB/s eta 0:00:0000:0100:01
  Using cached jinja2-3.1.5-py3-none-any.whl (134 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 

In [1]:
import os
import torch
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from datasets import load_dataset, Audio

In [2]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

In [6]:
train_path = Path('../Data/dataset/train')
files = os.listdir(train_path)
train_csv = [file for file in files if file.endswith('csv')]

In [7]:
trains, valids = [], []
for file in train_csv:
    df = pd.read_csv(train_path /file)[['audio_filepath', 'transcript']]
    df = df.rename(columns = {'audio_filepath': 'audio_path'})

    train_df, valid_df = train_test_split(df, test_size = 0.1, random_state = 42)
    
    trains.append(train_df)
    valids.append(valid_df)

In [8]:
valid = pd.concat(valids, axis = 0)
train = pd.concat(trains, axis = 0)

In [9]:
train.to_csv('train.csv', index = False)
valid.to_csv('val.csv', index = False)

In [3]:
# Define a function to process each example:
# It extracts the audio features and tokenizes the transcript.




In [9]:
def main():
    # Specify the model name (using the large version)
    model_name = "openai/whisper-large"
    
    # Load the processor and model from Hugging Face
    processor = WhisperProcessor.from_pretrained(model_name)
    model = WhisperForConditionalGeneration.from_pretrained(model_name)
    # Create a data collator to dynamically pad the input features and labels in each batch.
    def data_collator(features):
        # Extract input features and labels lists from the batch
        print(features)
        input_features = [f["input_features"] for f in features]
        input_features = {"input_features": input_features}
        labels = [f["labels"] for f in features]
        # Pad input features (using the feature extractor's padding method)`
        batch_inputs = processor.feature_extractor.pad(input_features, return_tensors="pt")
        
        # Pad labels using the tokenizer's pad method.
        batch_labels = processor.tokenizer.pad({"input_ids": labels}, return_tensors="pt")["input_ids"]
        
        return {"input_features": batch_inputs.input_features, "labels": batch_labels}
    def prepare_example(batch):
        # "audio_path" is automatically loaded as a dictionary with an "array" key.
        audio = batch["audio_path"]["array"]
        transcript = batch["transcript"]
    
        # Extract features from the audio using the processor's feature extractor.
        # The result is a list with one element per audio sample.
        input_features = processor.feature_extractor(audio, sampling_rate=16000).input_features[0]
        
        # Tokenize the transcript (the tokenizer will handle any necessary preprocessing).
        labels = processor.tokenizer(transcript).input_ids
    
        # Store the processed features and labels in the batch.
        batch["input_features"] = input_features
        batch["labels"] = labels
        return batch
    # Load your custom dataset from CSV files.
    # Ensure your CSV files have at least two columns: "audio_path" and "transcript".
    data_files = {"train": "train.csv", "validation": "val.csv"}
    dataset = load_dataset("csv", data_files=data_files)
    
    # Cast the "audio_path" column to an Audio column with the desired sampling rate.
    dataset = dataset.cast_column("audio_path", Audio(sampling_rate=16000))
    
    # Apply the preprocessing function to all examples.
    # remove_columns will drop the original columns so that the model receives only what it needs.
    dataset = dataset.map(prepare_example, remove_columns=dataset["train"].column_names)
    # Define the training arguments.
    # Adjust the batch sizes, learning rate, number of epochs, etc. according to your needs.
    training_args = Seq2SeqTrainingArguments(
        output_dir="./whisper-finetuned",
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        evaluation_strategy="steps",
        num_train_epochs=3,
        fp16=True,  # enable this if you have a GPU that supports half precision
        save_steps=500,
        eval_steps=500,
        logging_steps=100,
        learning_rate=1e-5,
        predict_with_generate=True,
        logging_dir="./logs",
    )
    
    # Create a Trainer for sequence-to-sequence training.
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        data_collator=data_collator,
        tokenizer=processor.tokenizer,
    )
    trainer.train()

In [10]:
from accelerate import notebook_launcher
notebook_launcher(finetuning_loop, num_processes=2)

Launching training on 2 GPUs.


/venv/main/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/venv/main/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_1494/2479828396.py:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/tmp/ipykernel_1494/2479828396.py:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To ch

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
[rank0]:[W228 09:27:42.926927369 reducer.cpp:1400] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused 

ChildFailedError: 
============================================================
finetuning_loop FAILED
------------------------------------------------------------
Failures:
  <NO_OTHER_FAILURES>
------------------------------------------------------------
Root Cause (first observed failure):
[0]:
  time      : 2025-02-28_09:27:42
  host      : fada9b19e18c
  rank      : 0 (local_rank: 0)
  exitcode  : 1 (pid: 1718)
  error_file: /tmp/torchelastic_ccu0ujml/none_79lkkrrl/attempt_0/0/error.json
  traceback : Traceback (most recent call last):
    File "/venv/main/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 355, in wrapper
      return f(*args, **kwargs)
    File "/tmp/ipykernel_1494/2479828396.py", line 75, in finetuning_loop
      trainer.train()
    File "/venv/main/lib/python3.10/site-packages/transformers/trainer.py", line 2241, in train
      return inner_training_loop(
    File "/venv/main/lib/python3.10/site-packages/transformer

In [23]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [11]:
os.environ["CUDA_VISIBLE_DEVICES"]

KeyError: 'CUDA_VISIBLE_DEVICES'

In [4]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Total GPUs: {torch.cuda.device_count()}")

for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)} - {torch.cuda.memory_allocated(i)/1024**3:.2f} GB allocated, {torch.cuda.memory_reserved(i)/1024**3:.2f} GB reserved")


PyTorch version: 2.6.0+cu124
CUDA available: True
Total GPUs: 2
GPU 0: NVIDIA GeForce RTX 4090 - 0.00 GB allocated, 0.00 GB reserved
GPU 1: NVIDIA GeForce RTX 4090 - 0.00 GB allocated, 0.00 GB reserved


In [5]:
torch.cuda.empty_cache()
import gc
gc.collect()


288